In [155]:
# install.packages("lubridate")
# install.packages("mltools")
# install.packages("data.table")

library(lubridate)
library(mltools)
library(data.table)


In [156]:
dados <- read.csv("../data/credito4.csv", sep = ";", na.strings = "", stringsAsFactors = TRUE, encoding = "UTF-8")

head(dados)


,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1169,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
2,5951,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
3,2096,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
4,7882,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
5,4870,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim
6,9055,4,35,nenhum,04/01/2020,masculino solteiro,educação,bom


In [157]:
# Verificando a quantidade de valores NA
summary(dados[!complete.cases(dados), ])


  SALDO_ATUAL   RESIDENCIADESDE     IDADE       OUTROSPLANOSPGTO         DATA  
 Min.   :1204   Min.   :1       Min.   :20.00   banco : 2        21/12/2019:2  
 1st Qu.:1278   1st Qu.:2       1st Qu.:27.50   nenhum:12        04/03/2019:1  
 Median :1938   Median :4       Median :35.00   stores: 1        08/01/2019:1  
 Mean   :2262   Mean   :3       Mean   :36.33                    13/01/2020:1  
 3rd Qu.:2667   3rd Qu.:4       3rd Qu.:42.00                    13/04/2020:1  
 Max.   :4771   Max.   :4       Max.   :63.00                    17/01/2018:1  
 NA's   :7                                                       (Other)   :8  
                 ESTADOCIVIL               PROPOSITO  CLASSE  
 fem div/cas           :3    radio/tv           :6   bom :10  
 masculino casado/viuvo:0    carro novo         :4   ruim: 5  
 masculino div/sep     :0    mobilia/equipamento:2            
 masculino solteiro    :4    carro usado        :1            
 NA's                  :8    Eletrodomésticos

### NA

In [158]:
summary(dados$SALDO_ATUAL)


     Min.   1st Qu.    Median      Mean   3rd Qu.      Max.      NA's 
2.500e+02 1.371e+03 2.323e+03 2.426e+07 3.976e+03 2.154e+10         7 

In [159]:
median(dados$SALDO_ATUAL, na.rm = TRUE)


[1] 2323

In [160]:
# Substituindo os valores NA pela mediana
dados$SALDO_ATUAL[is.na(dados$SALDO_ATUAL)] <- median(dados$SALDO_ATUAL, na.rm = TRUE)


In [161]:
dados[!complete.cases(dados$SALDO_ATUAL), ]


SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>


In [162]:
unique(dados$ESTADOCIVIL)


[1] masculino solteiro     fem div/cas            masculino div/sep     
[4] masculino casado/viuvo <NA>                  
4 Levels: fem div/cas masculino casado/viuvo ... masculino solteiro

In [163]:
summary(dados$ESTADOCIVIL)


fem div/cas masculino casado/viuvo      masculino div/sep 
                   308                     92                     50 
    masculino solteiro                   NA's 
                   542                      8

In [164]:
# Substituindo os valores NA por "masculino solteiro"
dados[is.na(dados$ESTADOCIVIL), ]$ESTADOCIVIL <- "masculino solteiro"


In [165]:
dados[is.na(dados$ESTADOCIVIL), ]


SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>


### Outliers

In [166]:
desv <- sd(dados$SALDO_ATUAL, na.rm = TRUE)
desv


[1] 685936689

In [167]:
# Exibindo os valores que estão acima de 2 desvios padrão
dados[dados$SALDO_ATUAL >= 2 * desv, ]$SALDO_ATUAL


[1]  2541111000 21544411255

In [168]:
# Substituindo os valores que estão acima de 2 desvios padrão pela mediana
dados[dados$SALDO_ATUAL > 2 * desv, ]$SALDO_ATUAL <- median(dados$SALDO_ATUAL, na.rm = TRUE)


In [169]:
dados[dados$SALDO_ATUAL > 2 * desv, ]$SALDO_ATUAL


numeric(0)

### Data Binding

In [170]:
summary(dados$PROPOSITO)


carro novo         carro usado            educação    Eletrodomésticos 
                234                 103                  50                  12 
mobilia/equipamento            negócios               obras              outros 
                181                  97                  22                  12 
       qualificação            radio/tv 
                  9                 280

In [171]:
dados[dados$PROPOSITO == "Eletrodomésticos", ]$PROPOSITO <- "outros"
dados[dados$PROPOSITO == "mobilia/equipamento", ]$PROPOSITO <- "outros"
dados[dados$PROPOSITO == "educação", ]$PROPOSITO <- "outros"


In [172]:
# Convertendo a variável PROPOSITO para fator
dados$PROPOSITO <- factor(dados$PROPOSITO)


In [173]:
summary(dados$PROPOSITO)


carro novo  carro usado     negócios        obras       outros qualificação 
         234          103           97           22          255            9 
    radio/tv 
         280

### Generation of attributes with Date

In [174]:
class(dados$DATA)


[1] "factor"

In [175]:
head(dados$DATA)


[1] 01/01/2019 01/01/2020 02/01/2020 02/01/2019 03/01/2018 04/01/2020
337 Levels: 01/01/2019 01/01/2020 01/02/2019 01/04/2018 ... 31/10/2020

In [176]:
dados$DATA <- as.Date(dmy(dados$DATA))

head(dados$DATA)


[1] "2019-01-01" "2020-01-01" "2020-01-02" "2019-01-02" "2018-01-03"
[6] "2020-01-04"

In [177]:
class(dados$DATA)


[1] "Date"

In [178]:
dados$ANO <- year(dados$DATA)


In [179]:
dados$MES <- month(dados$DATA)


In [180]:
dados$DIASEMANA <- weekdays(dados$DATA)

head(dados$DIASEMANA)


[1] "Tuesday"   "Wednesday" "Thursday"  "Wednesday" "Wednesday" "Saturday"

In [181]:
head(dados)


,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE,ANO,MES,DIASEMANA
,<dbl>,<int>,<int>,<fct>,<date>,<fct>,<fct>,<fct>,<int>,<int>,<chr>
1,1169,4,67,nenhum,2019-01-01,masculino solteiro,radio/tv,bom,2019,1,Tuesday
2,5951,2,22,nenhum,2020-01-01,fem div/cas,radio/tv,ruim,2020,1,Wednesday
3,2096,3,49,nenhum,2020-01-02,masculino solteiro,outros,bom,2020,1,Thursday
4,7882,4,45,nenhum,2019-01-02,masculino solteiro,outros,bom,2019,1,Wednesday
5,4870,4,53,nenhum,2018-01-03,masculino solteiro,carro novo,ruim,2018,1,Wednesday
6,9055,4,35,nenhum,2020-01-04,masculino solteiro,outros,bom,2020,1,Saturday


### Label Encoder

In [182]:
unique(dados$ESTADOCIVIL)


[1] masculino solteiro     fem div/cas            masculino div/sep     
[4] masculino casado/viuvo
4 Levels: fem div/cas masculino casado/viuvo ... masculino solteiro

In [183]:
# Convertendo a variável ESTADOCIVIL para numérica
dados$ESTADOCIVIL <- as.numeric(dados$ESTADOCIVIL)

head(dados$ESTADOCIVIL)


[1] 4 1 4 4 4 4

In [184]:
unique(dados$PROPOSITO)


[1] radio/tv     outros       carro novo   carro usado  negócios    
[6] obras        qualificação
7 Levels: carro novo carro usado negócios obras outros ... radio/tv

In [185]:
dados$PROPOSITO <- as.numeric(dados$PROPOSITO)


In [186]:
dados$DIASEMANA <- as.factor(dados$DIASEMANA)
dados$DIASEMANA <- as.numeric(dados$DIASEMANA)


In [187]:
head(dados)


,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE,ANO,MES,DIASEMANA
,<dbl>,<int>,<int>,<fct>,<date>,<dbl>,<dbl>,<fct>,<int>,<int>,<dbl>
1,1169,4,67,nenhum,2019-01-01,4,7,bom,2019,1,6
2,5951,2,22,nenhum,2020-01-01,1,7,ruim,2020,1,7
3,2096,3,49,nenhum,2020-01-02,4,5,bom,2020,1,5
4,7882,4,45,nenhum,2019-01-02,4,5,bom,2019,1,7
5,4870,4,53,nenhum,2018-01-03,4,1,ruim,2018,1,7
6,9055,4,35,nenhum,2020-01-04,4,5,bom,2020,1,3


### One-hot Encoding

In [188]:
# Convertendo a variável OUTROSPLANOSPGTO para one-hot encoding
dados_temp <- one_hot(as.data.table(dados$OUTROSPLANOSPGTO))

head(dados_temp)


V1_banco,V1_nenhum,V1_stores
<int>,<int>,<int>
0,1,0
0,1,0
0,1,0
0,1,0
0,1,0
0,1,0


In [189]:
# Adicionando as variáveis dummies ao conjunto de dados
dados <- cbind(dados, dados_temp)

head(dados)

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE,ANO,MES,DIASEMANA,V1_banco,V1_nenhum,V1_stores
,<dbl>,<int>,<int>,<fct>,<date>,<dbl>,<dbl>,<fct>,<int>,<int>,<dbl>,<int>,<int>,<int>
1,1169,4,67,nenhum,2019-01-01,4,7,bom,2019,1,6,0,1,0
2,5951,2,22,nenhum,2020-01-01,1,7,ruim,2020,1,7,0,1,0
3,2096,3,49,nenhum,2020-01-02,4,5,bom,2020,1,5,0,1,0
4,7882,4,45,nenhum,2019-01-02,4,5,bom,2019,1,7,0,1,0
5,4870,4,53,nenhum,2018-01-03,4,1,ruim,2018,1,7,0,1,0
6,9055,4,35,nenhum,2020-01-04,4,5,bom,2020,1,3,0,1,0


### Normalizing data

In [190]:
p <- as.data.frame(scale(dados[1:3]))
colnames(p) <- c("SALDO_ATUAL_P", "RESISDENCIADESDE_P", "IDADE_P")
head(p)


,SALDO_ATUAL_P,RESISDENCIADESDE_P,IDADE_P
,<dbl>,<dbl>,<dbl>
1,-0.7451436,1.0464631,1.63845605
2,0.9572614,-0.7655942,-0.73987118
3,-0.4151291,0.1404344,0.68712516
4,1.6447026,1.0464631,0.47571830
5,0.5724225,1.0464631,0.89853203
6,2.0622938,1.0464631,-0.05279886


In [191]:
# Removendo as variáveis originais
dados <- cbind(p, dados[4:ncol(dados)])

head(dados)

,SALDO_ATUAL_P,RESISDENCIADESDE_P,IDADE_P,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE,ANO,MES,DIASEMANA,V1_banco,V1_nenhum,V1_stores
,<dbl>,<dbl>,<dbl>,<fct>,<date>,<dbl>,<dbl>,<fct>,<int>,<int>,<dbl>,<int>,<int>,<int>
1,-0.7451436,1.0464631,1.63845605,nenhum,2019-01-01,4,7,bom,2019,1,6,0,1,0
2,0.9572614,-0.7655942,-0.73987118,nenhum,2020-01-01,1,7,ruim,2020,1,7,0,1,0
3,-0.4151291,0.1404344,0.68712516,nenhum,2020-01-02,4,5,bom,2020,1,5,0,1,0
4,1.6447026,1.0464631,0.47571830,nenhum,2019-01-02,4,5,bom,2019,1,7,0,1,0
5,0.5724225,1.0464631,0.89853203,nenhum,2018-01-03,4,1,ruim,2018,1,7,0,1,0
6,2.0622938,1.0464631,-0.05279886,nenhum,2020-01-04,4,5,bom,2020,1,3,0,1,0


In [192]:
# Removendo as variáveis originais

# SALDO_ATUAL, RESIDENCIADESDE, IDADE - Padronização
# OUTROSPLANOSPGTO - One-hot encoding
# DATA - Ano, Mês, Dia da semana
# V1_banco - Dummy variable trap

df2 <- dados[!names(dados) %in% c("OUTROSPLANOSPGTO", "DATA", "V1_banco")]

head(df2)


,SALDO_ATUAL_P,RESISDENCIADESDE_P,IDADE_P,ESTADOCIVIL,PROPOSITO,CLASSE,ANO,MES,DIASEMANA,V1_nenhum,V1_stores
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<dbl>,<int>,<int>
1,-0.7451436,1.0464631,1.63845605,4,7,bom,2019,1,6,1,0
2,0.9572614,-0.7655942,-0.73987118,1,7,ruim,2020,1,7,1,0
3,-0.4151291,0.1404344,0.68712516,4,5,bom,2020,1,5,1,0
4,1.6447026,1.0464631,0.47571830,4,5,bom,2019,1,7,1,0
5,0.5724225,1.0464631,0.89853203,4,1,ruim,2018,1,7,1,0
6,2.0622938,1.0464631,-0.05279886,4,5,bom,2020,1,3,1,0
